# Boston Housing - Price Prediction with Amazon SageMaker

In this module we will use the same dataset and model, but will update the model to use features of SageMaker to scale dataset transformation and model training beyond Jupyter notebook.

This notebook includes all key steps such as preprocessing data with SageMaker Processing, and model training and deployment with SageMaker hosted training and inference. Automatic Model Tuning in SageMaker is used to tune the model's hyperparameters. If you are using TensorFlow 2, you can use the Amazon SageMaker prebuilt TensorFlow 2 framework container with training scripts similar to those you would use outside SageMaker.

In this first notebook we will predict house prices based on the well-known Boston Housing dataset with a simple regression model in Tensorflow 2. This public dataset contains 13 features regarding housing stock of towns in the Boston area.  Features include average number of rooms, accessibility to radial highways, adjacency to a major river, etc.  

To begin, we'll import some necessary packages and set up directories for training and test data.  We'll also set up a SageMaker Session to perform various operations, and specify an Amazon S3 bucket to hold input data and output.  The default bucket used here is created by SageMaker if it doesn't already exist, and named in accordance with the AWS account ID and AWS Region.  

## Data preprocessing

Next, we'll import the dataset and transform it with SageMaker Processing, which can be used to process terabytes of data in a SageMaker-managed cluster separate from the instance running your notebook server. In a typical SageMaker workflow, notebooks are only used for prototyping and can be run on relatively inexpensive and less powerful instances, while processing, training and model hosting tasks are run on separate, more powerful SageMaker-managed instances.  SageMaker Processing includes off-the-shelf support for Scikit-learn, as well as a Bring Your Own Container option, so it can be used with many different data transformation technologies and tasks.  An alternative to SageMaker Processing is [SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/), a visual data preparation tool integrated with the SageMaker Studio UI.    

To work with SageMaker Processing, first we'll load the Boston Housing dataset, save the raw feature data and upload it to Amazon S3 so it can be accessed by SageMaker Processing.  We'll also save the labels for training and testing.

In [ ]:
import os
import boto3
import sagemaker

session = sagemaker.session.Session()

data_dir = os.path.join(os.getcwd(), 'data')
os.makedirs(data_dir, exist_ok=True)

In [ ]:
import numpy as np
from sklearn.datasets import load_boston

X, y = load_boston(return_X_y=True)

np.save(os.path.join(data_dir, 'X.npy'), X)
np.save(os.path.join(data_dir, 'y.npy'), y)

raw_dataset = session.upload_data(path=data_dir, key_prefix='boston-housing/data/raw')

In [ ]:
print(raw_dataset)

Next, simply supply an ordinary Python data preprocessing script as shown below.  For this example, we're using a SageMaker prebuilt Scikit-learn framework container, which includes many common functions for processing data.  There are few limitations on what kinds of code and operations you can run, and only a minimal API contract:  input and output data must be placed in specified directories.  If this is done, SageMaker Processing automatically loads the input data from S3 and uploads transformed data back to S3 when the job is complete.

Before starting the SageMaker Processing job, we instantiate a `SKLearnProcessor` object.  This object allows you to specify the instance type to use in the job, as well as how many instances.  Spinning a cluster is just a matter of setting `instahce_count` to 2 or more, but our transformation has a `StandardScaler` which must be run over all training data and applied equally to train and test data. That can't be parallelized with `scikit-learn`, but since the dataset is small, that is not a problem.

In [ ]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

execution_role = get_execution_role()

sklearn_processor = SKLearnProcessor(framework_version='0.23-1',
                                     role=execution_role,
                                     instance_type='ml.m5.xlarge',
                                     instance_count=1)

We're now ready to run the Processing job.  To enable distributing the data files equally among the instances, we specify the `ShardedByS3Key` distribution type in the `ProcessingInput` object.  This ensures that if we have `n` instances, each instance will receive `1/n` files from the specified S3 bucket.  It may take around 3 minutes for the following code cell to run, mainly to set up the cluster.  At the end of the job, the cluster automatically will be torn down by SageMaker.  

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

bucket = session.default_bucket() 

processed_dataset = 's3://{}/boston-housing/data/processed'.format(bucket)

sklearn_processor.run(
    code='code/preprocessing/preprocessing.py',
    inputs=[ProcessingInput(
        source=raw_dataset,
        destination='/opt/ml/processing/input'
    )],
    outputs=[ProcessingOutput(
        source='/opt/ml/processing/output',
        destination=processed_dataset
    )]
)

In the log output of the SageMaker Processing job above, you should be able to see logs in two different colors for the two different instances, and that each instance received different files.  Without the `ShardedByS3Key` distribution type, each instance would have received a copy of **all** files.  By spreading the data equally among `n` instances, you should receive a speedup by approximately a factor of `n` for most stateless data transformations.  After saving the job results locally, we'll move on to training and inference code.

## Training

Now that we've prepared a dataset, we can move on to SageMaker's model training functionality. With SageMaker hosted training the actual training itself occurs not on the notebook instance, but on a separate cluster of machines managed by SageMaker. Before starting hosted training, the data must be in S3, or an EFS or FSx for Lustre file system. We'll upload to S3 now, and confirm the upload was successful.

We're now ready to set up an Estimator object for hosted training. We simply call `fit` to start the actual hosted training.

In [ ]:
from sagemaker.tensorflow import TensorFlow

hyperparameters = {'epochs': 50, 'batch_size': 128, 'learning_rate': 0.01}

hosted_estimator = TensorFlow(
                       source_dir='code/train',
                       entry_point='train.py',
                       instance_type='ml.c5.xlarge',
                       instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       framework_version='2.3.1',
                       py_version='py37')

After starting the hosted training job with the `fit` method call below, you should observe the valication loss converge with each epoch.  Can we do better? We'll look into a way to do so in the **Automatic Model Tuning** section below. In the meantime, the hosted training job should take about 3 minutes to complete.  

In [ ]:
hosted_estimator.fit({'input':processed_dataset})

The training job produces a model saved in S3 that we can retrieve.  This is an example of the modularity of SageMaker: having trained the model in SageMaker, you can now take the model out of SageMaker and run it anywhere else.  Alternatively, you can deploy the model into a production-ready environment using SageMaker's hosted endpoints functionality, as shown in the **SageMaker hosted endpoint** section below.

Retrieving the model from S3 is very easy:  the hosted training estimator you created above stores a reference to the model's location in S3.  You simply copy the model from S3 using the estimator's `model_data` property and unzip it to inspect the contents.

In [ ]:
!aws s3 cp {hosted_estimator.model_data} ./model/model.tar.gz

The unzipped archive should include the assets required by TensorFlow Serving to load the model and serve it, including a .pb file:  

In [ ]:
!tar -xvzf ./model/model.tar.gz -C ./model

## Validation
    
The final step in this pipeline is offline, batch scoring (inference/prediction).  The inputs to this step will be the model we trained earlier, and the test data.  A simple, ordinary Python script is all we need to do the actual batch inference.

In [ ]:
batch_input = 's3://{}/boston-housing/data/processed/X_val.json'.format(bucket)
batch_output = 's3://{}/boston-housing/batch-output'.format(bucket)

tf_transformer = hosted_estimator.transformer(
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=batch_output
)

tf_transformer.transform(data=batch_input, content_type='application/json')

In [ ]:
!aws s3 cp --recursive $tf_transformer.output_path ./

In [ ]:
!aws s3 cp s3://$bucket/boston-housing/data/processed/y_val.json ./

In [ ]:
import json 

with open("X_val.json.out", "r") as read_file:
    validation_pred = json.load(read_file)
    validation_pred = [item for item_arr in validation_pred['predictions'] for item in item_arr]

with open("y_val.json", "r") as read_file:
    validation_true = json.load(read_file)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(validation_true, validation_pred)
print('Validation MSE: {}'.format(mse))

## Automatic Model Tuning

So far we have simply run one Hosted Training job without any real attempt to tune hyperparameters to produce a better model.  Selecting the right hyperparameter values to train your model can be difficult, and typically is very time consuming if done manually. The right combination of hyperparameters is dependent on your data and algorithm; some algorithms have many different hyperparameters that can be tweaked; some are very sensitive to the hyperparameter values selected; and most have a non-linear relationship between model fit and hyperparameter values.  SageMaker Automatic Model Tuning helps automate the hyperparameter tuning process:  it runs multiple training jobs with different hyperparameter combinations to find the set with the best model performance.

We begin by specifying the hyperparameters we wish to tune, and the range of values over which to tune each one.  We also must specify an objective metric to be optimized:  in this use case, we'd like to minimize the validation loss.

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
  'learning_rate': ContinuousParameter(0.001, 0.2, scaling_type="Logarithmic"),
  'epochs': IntegerParameter(20, 90),
  'batch_size': IntegerParameter(64, 256),
}

metric_definitions = [{'Name': 'loss',
                       'Regex': ' loss: ([0-9\\.]+)'},
                     {'Name': 'val_loss',
                       'Regex': ' val_loss: ([0-9\\.]+)'}]

objective_metric_name = 'val_loss'
objective_type = 'Minimize'

Next we specify a HyperparameterTuner object that takes the above definitions as parameters.  Each tuning job must be given a budget:  a maximum number of training jobs.  A tuning job will complete after that many training jobs have been executed.  

We also can specify how much parallelism to employ, in this case five jobs, meaning that the tuning job will complete after three series of five jobs in parallel have completed.  For the default Bayesian Optimization tuning strategy used here, the tuning search is informed by the results of previous groups of training jobs, so we don't run all of the jobs in parallel, but rather divide the jobs into groups of parallel jobs.  There is a trade-off: using more parallel jobs will finish tuning sooner, but likely will sacrifice tuning search accuracy. 

Now we can launch a hyperparameter tuning job by calling the `fit` method of the HyperparameterTuner object.  The tuning job may take around 10 minutes to finish.  While you're waiting, the status of the tuning job, including metadata and results for invidual training jobs within the tuning job, can be checked in the SageMaker console in the **Hyperparameter tuning jobs** panel.  

In [ ]:
tuner = HyperparameterTuner(hosted_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=15,
                            max_parallel_jobs=5,
                            objective_type=objective_type)

tuner.fit({'input':processed_dataset}, job_name='boston-housing-tuning')
tuner.wait()

After the tuning job is finished, we can use the `HyperparameterTuningJobAnalytics` object from the SageMaker Python SDK to list the top 5 tuning jobs with the best performance. Although the results vary from tuning job to tuning job, the best validation loss from the tuning job (under the FinalObjectiveValue column) likely will be substantially lower than the validation loss from the hosted training job above.  

In [ ]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics('boston-housing-tuning')
tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=True).head(5)

The total training time and training jobs status can be checked with the following lines of code. Because automatic early stopping is by default off, all the training jobs should be completed normally.  For an example of a more in-depth analysis of a tuning job, see the SageMaker official sample [HPO_Analyze_TuningJob_Results.ipynb](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb) notebook.

In [ ]:
total_time = tuner_metrics.dataframe()['TrainingElapsedTimeSeconds'].sum() / 3600
print("The total training time is {:.2f} hours".format(total_time))
tuner_metrics.dataframe()['TrainingJobStatus'].value_counts()

## Endpoint deployment

Assuming the best model from the tuning job is better than the model produced by the individual hosted training job above, we could now easily deploy that model to production.  A convenient option is to use a SageMaker hosted endpoint, which serves real time predictions from the trained model (For asynchronous, offline predictions on large datasets, you can use either SageMaker Processing or SageMaker Batch Transform.). The endpoint will retrieve the TensorFlow SavedModel created during training and deploy it within a SageMaker TensorFlow Serving container. This all can be accomplished with one line of code.  

More specifically, by calling the `deploy` method of the HyperparameterTuner object we instantiated above, we can directly deploy the best model from the tuning job to a SageMaker hosted endpoint.

In [ ]:
!aws s3 cp s3://$bucket/boston-housing/data/processed/StandardScaler.pkl ./code/

In [ ]:
best_model = tuner.best_estimator().create_model(entry_point='inference.py',
              source_dir='code/inference')

In [ ]:
tuning_predictor = best_model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

We can compare the predictions generated by this endpoint with the actual target values: 

In [ ]:
results = tuning_predictor.predict({'instances': X[:10]})['predictions']

flat_list = [float('%.1f'%(item)) for sublist in results for item in sublist]

print('predictions: \t{}'.format(np.array(flat_list)))
print('target values: \t{}'.format(y[:10].round(decimals=1)))

#### How would the code look like in your application? 

Here is a sample function that accepts property parameters as function input and returns predicted property value:

In [ ]:
def predict_house_value(crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, b, lstat):
    """Predict housing value in $1000's

    Parameters
    ----------
    crim : float
        per capita crime rate by town
    zn : float
        proportion of residential land zoned for lots over 25,000 sq.ft.
    indus : float
        proportion of non-retail business acres per town
    chas : int
        Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
    nox : float
        nitric oxides concentration (parts per 10 million)
    rm : float
        average number of rooms per dwelling
    age : float
        proportion of owner-occupied units built prior to 1940
    dis : float
        weighted distances to five Boston employment centres
    rad : float
        index of accessibility to radial highways
    tax : float
        full-value property-tax rate per $10,000
    ptratio : float
        pupil-teacher ratio by town
    b : float
        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    lstat : float
        % lower status of the population

    Returns
    -------
    float
        Median value of owner-occupied homes in $1000’s
    """
    model_input = {
        'instances': [
            [crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, b, lstat]
        ]
    }

    return tuning_predictor.predict(model_input)['predictions'][0][0]


In [ ]:
predicted_value = predict_house_value(0.01, 18.0, 2.31, 0, 0.54, 6.58, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98)
print('Predicted house value: ${:.2f}k'.format(predicted_value))